In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import vgg19
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
from scipy.optimize import fmin_l_bfgs_b
from PIL import Image

# Step 1: Load and preprocess the content and style images
def load_and_preprocess_img(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

# Load your images
content_image_path = 'path_to_your_content_image.jpg'
style_image_path = 'path_to_your_style_image.jpg'
content_image = load_and_preprocess_img(content_image_path)
style_image = load_and_preprocess_img(style_image_path)

# Step 2: Define the model
# Load the VGG19 model with weights pre-trained on ImageNet
model = vgg19.VGG19(include_top=False, weights='imagenet')

# Define the layers we'll need
output_layers = {
    'content': 'block5_conv2',  # Content layer
    'style': ['block1_conv1',   # Style layers
              'block2_conv1',
              'block3_conv1',
              'block4_conv1',
              'block5_conv1']
}

# Create models for content and style features
content_model = Model(inputs=model.input, outputs=model.get_layer(output_layers['content']).output)
style_models = [Model(inputs=model.input, outputs=model.get_layer(layer).output) for layer in output_layers['style']]

# Step 3: Define the loss functions
def content_loss(content, combination):
    return tf.reduce_sum(tf.square(combination - content))

def gram_matrix(input_tensor):
    features = tf.reshape(input_tensor, (-1, input_tensor.shape[3]))
    gram = tf.matmul(features, tf.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = style_image.shape[1] * style_image.shape[2]
    return tf.reduce_sum(tf.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

# Step 4: Training loop
def compute_loss_and_grads(content_image, style_image, combination_image):
    with tf.GradientTape() as tape:
        # Extract features of the content, style, and combination images
        content_features = content_model(content_image)
        style_features = [model(style_image) for model in style_models]
        combination_features = content_model(combination_image)
        comb_style_features = [model(combination_image) for model in style_models]

        # Initialize the loss
        loss = tf.zeros(shape=())

        # Add content loss
        loss += content_loss(content_features, combination_features) * content_weight

        # Add style loss
        for comb_feat, style_feat in zip(comb_style_features, style_features):
            loss += (style_loss(style_feat, comb_feat) * style_weight) / len(output_layers['style'])

    # Compute gradients
    grads = tape.gradient(loss, combination_image)
    return loss, grads

# Step 5: Optimization
def style_transfer(content_image, style_image, num_iterations=10):
    combination_image = tf.Variable(content_image, dtype=tf.float32)
    optimizer = tf.optimizers.Adam(learning_rate=5.0)

    for i in range(num_iterations):
        loss, grads = compute_loss_and_grads(content_image, style_image, combination_image)
        optimizer.apply_gradients([(grads, combination_image)])
        if i % 1 == 0:
            print(f'Iteration {i}: loss={loss}')
            img = deprocess_img(combination_image.numpy())
            plt.imshow(img)
            plt.show()

# Step 6: Run the style transfer
style_transfer(content_image, style_image, num_iterations=10)

# Utility function to convert a tensor into a valid image
def deprocess_img(processed_img):
    x = processed_img.copy()
    if len(x.shape) == 4:
        x = np.squeeze(x, 0)
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x
